In [62]:
import threading
import time
import random
from collections import defaultdict

In [63]:
def safe_print(*args, **kwargs):
    print(*args, **kwargs, flush=True)

In [64]:
# ==================== Базовий клас мережі Петрі ====================
class PetriNet:
    def __init__(self):
        self.places = {}  # {place_name: token_count}
        self.transitions = {}  # {transition_name: {'input': [(place, weight)], 'output': [(place, weight)]}}
        self.lock = threading.Lock()
    
    def add_place(self, name, tokens=0):
        self.places[name] = tokens
    
    def add_transition(self, name, inputs, outputs):
        """
        inputs: [(place_name, weight), ...]
        outputs: [(place_name, weight), ...]
        """
        self.transitions[name] = {'input': inputs, 'output': outputs}
    
    def is_enabled(self, transition_name):
        """Перевірка чи дозволений перехід"""
        trans = self.transitions[transition_name]
        for place, weight in trans['input']:
            if self.places.get(place, 0) < weight:
                return False
        return True
    
    def fire(self, transition_name):
        """Спрацювання переходу"""
        with self.lock:
            if not self.is_enabled(transition_name):
                return False
            
            trans = self.transitions[transition_name]
            
            # Забрати фішки з вхідних позицій
            for place, weight in trans['input']:
                self.places[place] -= weight
            
            # Додати фішки до вихідних позицій
            for place, weight in trans['output']:
                self.places[place] = self.places.get(place, 0) + weight
            
            return True
    
    def get_marking(self):
        """Отримати поточну розмітку"""
        return dict(self.places)
    
    def print_state(self):
        safe_print("Поточна розмітка:", self.get_marking())

1. Задача про обідаючих філософів. П'ять філософів відпочивають у пансіонаті. Кожен із них 
може або вдаватися до роздумів, або обідати, відвідавши їдальню. У їдальні знаходиться 
круглий стіл, за яким є місця для кожного з філософів, п'ять тарілок, п'ять виделок, 
розташованих між тарілками, та страва спагетті у центрі столу. Кожен філософ для того щоб 
пообідати бере одночасно дві вилки і приступає до трапези, по завершенню якої він кладе 
вилки на стіл і уходить, повертаючись до роздумів.

In [65]:
class DiningPhilosophers1:
    def __init__(self, n=5):
        self.n = n
        self.pn = PetriNet()
        
        for i in range(n):
            self.pn.add_place(f"thinking_{i}", 1)
            self.pn.add_place(f"eating_{i}", 0)
            self.pn.add_place(f"fork_{i}", 1)
        
        for i in range(n):
            left_fork = i
            right_fork = (i + 1) % n
            
            self.pn.add_transition(
                f"take_{i}",
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)],
                [(f"eating_{i}", 1)]
            )
            
            self.pn.add_transition(
                f"release_{i}",
                [(f"eating_{i}", 1)],
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)]
            )
    
    def philosopher(self, phil_id):
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.3))
            
            while not self.pn.fire(f"take_{phil_id}"):
                time.sleep(0.05)
            
            print(f"Філософ {phil_id} їсть")
            time.sleep(random.uniform(0.1, 0.2))
            
            self.pn.fire(f"release_{phil_id}")
            print(f"Філософ {phil_id} закінчив їсти")
    
    def run(self):
        threads = [threading.Thread(target=self.philosopher, args=(i,)) for i in range(self.n)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

dp1 = DiningPhilosophers1(5)
dp1.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 0 їсть
Філософ 2 їсть
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 їсть
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 1 їсть
Філософ 4 закінчив їсти
Філософ 4 роздумує...
Філософ 3 їсть
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 0 їсть
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 2 їсть
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 4 їсть
Філософ 1 їсть
Філософ 4 закінчив їсти
Філософ 4 роздумує...
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 3 їсть
Філософ 0 їсть
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 2 їсть
Філософ 0 закінчив їсти
Філософ 4 їсть
Філософ 2 закінчив їсти
Філософ 1 їсть
Філософ 4 закінчив їсти
Філософ 3 їсть
Філософ 1 закінчив їсти
Філософ 3 закінчив їсти


2. Перетворення задачі 1 про обідаючих філософів для випадку, коли ліву та праву вилки 
беруть по черзі (не одночасно).

In [66]:
class DiningPhilosophers2:
    def __init__(self, n=5):
        self.n = n
        self.pn = PetriNet()
        
        for i in range(n):
            self.pn.add_place(f"thinking_{i}", 1)
            self.pn.add_place(f"has_left_{i}", 0)
            self.pn.add_place(f"eating_{i}", 0)
            self.pn.add_place(f"fork_{i}", 1)
        
        for i in range(n):
            left_fork = i
            right_fork = (i + 1) % n
            
            self.pn.add_transition(
                f"take_left_{i}",
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1)],
                [(f"has_left_{i}", 1)]
            )
            
            self.pn.add_transition(
                f"take_right_{i}",
                [(f"has_left_{i}", 1), (f"fork_{right_fork}", 1)],
                [(f"eating_{i}", 1)]
            )
            
            self.pn.add_transition(
                f"release_{i}",
                [(f"eating_{i}", 1)],
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)]
            )
    
    def philosopher(self, phil_id):
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.3))
            
            while not self.pn.fire(f"take_left_{phil_id}"):
                time.sleep(0.05)
            print(f"Філософ {phil_id} взяв ліву виделку")
            
            while not self.pn.fire(f"take_right_{phil_id}"):
                time.sleep(0.05)
            print(f"Філософ {phil_id} взяв праву виделку та їсть")
            
            time.sleep(random.uniform(0.1, 0.2))
            
            self.pn.fire(f"release_{phil_id}")
            print(f"Філософ {phil_id} закінчив їсти")
    
    def run(self):
        threads = [threading.Thread(target=self.philosopher, args=(i,)) for i in range(self.n)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

dp2 = DiningPhilosophers2(5)
dp2.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 0 взяв ліву виделку
Філософ 0 взяв праву виделку та їсть
Філософ 3 взяв ліву виделку
Філософ 3 взяв праву виделку та їсть
Філософ 2 взяв ліву виделку
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 1 взяв ліву виделку
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 2 взяв праву виделку та їсть
Філософ 4 взяв ліву виделку
Філософ 4 взяв праву виделку та їсть
Філософ 4 закінчив їсти
Філософ 4 роздумує...
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 1 взяв праву виделку та їсть
Філософ 3 взяв ліву виделку
Філософ 3 взяв праву виделку та їсть
Філософ 0 взяв ліву виделку
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 4 взяв ліву виделку
Філософ 0 взяв праву виделку та їсть
Філософ 2 взяв ліву виделку
Філософ 2 взяв праву виделку та їсть
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 0 закінчив ї

3. Перетворення задачі 2 для випадку, коли за стіл запрошують одного з філософів, який не 
перешкоджає загальному поглинанню їжі обідаючими.

In [67]:
class DiningPhilosophers3:
    def __init__(self, n=5):
        self.n = n
        self.pn = PetriNet()
        
        self.pn.add_place("invitation", 1)  # Запрошення до столу
        
        for i in range(n):
            self.pn.add_place(f"thinking_{i}", 1)
            self.pn.add_place(f"invited_{i}", 0)
            self.pn.add_place(f"eating_{i}", 0)
            self.pn.add_place(f"fork_{i}", 1)
        
        for i in range(n):
            left_fork = i
            right_fork = (i + 1) % n
            
            # Отримати запрошення
            self.pn.add_transition(
                f"get_invitation_{i}",
                [(f"thinking_{i}", 1), ("invitation", 1)],
                [(f"invited_{i}", 1)]
            )
            
            # Взяти виделки і їсти
            self.pn.add_transition(
                f"take_forks_{i}",
                [(f"invited_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)],
                [(f"eating_{i}", 1)]
            )
            
            # Покласти виделки і повернути запрошення
            self.pn.add_transition(
                f"release_{i}",
                [(f"eating_{i}", 1)],
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1), ("invitation", 1)]
            )
    
    def philosopher(self, phil_id):
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.3))
            
            while not self.pn.fire(f"get_invitation_{phil_id}"):
                time.sleep(0.05)
            print(f"Філософ {phil_id} отримав запрошення")
            
            while not self.pn.fire(f"take_forks_{phil_id}"):
                time.sleep(0.05)
            print(f"Філософ {phil_id} їсть")
            
            time.sleep(random.uniform(0.1, 0.2))
            
            self.pn.fire(f"release_{phil_id}")
            print(f"Філософ {phil_id} закінчив їсти та повернув запрошення")
    
    def run(self):
        threads = [threading.Thread(target=self.philosopher, args=(i,)) for i in range(self.n)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

dp3 = DiningPhilosophers3(5)
dp3.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 3 отримав запрошення
Філософ 3 їсть
Філософ 3 закінчив їсти та повернув запрошення
Філософ 3 роздумує...
Філософ 2 отримав запрошення
Філософ 2 їсть
Філософ 2 закінчив їсти та повернув запрошення
Філософ 2 роздумує...
Філософ 3 отримав запрошення
Філософ 3 їсть
Філософ 3 закінчив їсти та повернув запрошення
Філософ 3 роздумує...
Філософ 1 отримав запрошення
Філософ 1 їсть
Філософ 1 закінчив їсти та повернув запрошення
Філософ 1 роздумує...
Філософ 0 отримав запрошення
Філософ 0 їсть
Філософ 0 закінчив їсти та повернув запрошення
Філософ 0 роздумує...
Філософ 3 отримав запрошення
Філософ 3 їсть
Філософ 3 закінчив їсти та повернув запрошення
Філософ 1 отримав запрошення
Філософ 1 їсть
Філософ 1 закінчив їсти та повернув запрошення
Філософ 1 роздумує...
Філософ 0 отримав запрошення
Філософ 0 їсть
Філософ 0 закінчив їсти та повернув запрошення
Філософ 0 роздумує...
Філософ 

4. Перетворення задачі 3 для випадку, коли запрошення до столу необов'язкові для розгляду 
філософом.

In [68]:
class DiningPhilosophers4:
    def __init__(self, n=5):
        self.n = n
        self.pn = PetriNet()
        
        self.pn.add_place("invitation", 1)
        
        for i in range(n):
            self.pn.add_place(f"thinking_{i}", 1)
            self.pn.add_place(f"invited_{i}", 0)
            self.pn.add_place(f"eating_{i}", 0)
            self.pn.add_place(f"fork_{i}", 1)
        
        for i in range(n):
            left_fork = i
            right_fork = (i + 1) % n
            
            # Отримати запрошення
            self.pn.add_transition(
                f"get_invitation_{i}",
                [(f"thinking_{i}", 1), ("invitation", 1)],
                [(f"invited_{i}", 1)]
            )
            
            # Взяти виделки після запрошення
            self.pn.add_transition(
                f"take_forks_invited_{i}",
                [(f"invited_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)],
                [(f"eating_{i}", 1)]
            )
            
            # Взяти виделки БЕЗ запрошення (якщо філософ ігнорує запрошення)
            self.pn.add_transition(
                f"take_forks_direct_{i}",
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)],
                [(f"eating_{i}", 1)]
            )
            
            # Покласти виделки після запрошеної їжі
            self.pn.add_transition(
                f"release_invited_{i}",
                [(f"eating_{i}", 1)],
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1), ("invitation", 1)]
            )
            
            # Покласти виделки після прямої їжі
            self.pn.add_transition(
                f"release_direct_{i}",
                [(f"eating_{i}", 1)],
                [(f"thinking_{i}", 1), (f"fork_{left_fork}", 1), (f"fork_{right_fork}", 1)]
            )
    
    def philosopher(self, phil_id):
        for _ in range(3):
            print(f"Філософ {phil_id} роздумує...")
            time.sleep(random.uniform(0.1, 0.3))
            
            # Філософ вирішує: чекати запрошення чи їсти без нього
            wait_invitation = random.choice([True, False])
            
            if wait_invitation:
                while not self.pn.fire(f"get_invitation_{phil_id}"):
                    time.sleep(0.05)
                print(f"Філософ {phil_id} отримав запрошення")
                
                while not self.pn.fire(f"take_forks_invited_{phil_id}"):
                    time.sleep(0.05)
                print(f"Філософ {phil_id} їсть (з запрошенням)")
                
                time.sleep(random.uniform(0.1, 0.2))
                self.pn.fire(f"release_invited_{phil_id}")
                print(f"Філософ {phil_id} закінчив їсти та повернув запрошення")
            else:
                while not self.pn.fire(f"take_forks_direct_{phil_id}"):
                    time.sleep(0.05)
                print(f"Філософ {phil_id} їсть (БЕЗ запрошення)")
                
                time.sleep(random.uniform(0.1, 0.2))
                self.pn.fire(f"release_direct_{phil_id}")
                print(f"Філософ {phil_id} закінчив їсти")
    
    def run(self):
        threads = [threading.Thread(target=self.philosopher, args=(i,)) for i in range(self.n)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

dp4 = DiningPhilosophers4(5)
dp4.run()

Філософ 0 роздумує...
Філософ 1 роздумує...
Філософ 2 роздумує...
Філософ 3 роздумує...
Філософ 4 роздумує...
Філософ 0 їсть (БЕЗ запрошення)
Філософ 4 отримав запрошення
Філософ 2 їсть (БЕЗ запрошення)
Філософ 0 закінчив їсти
Філософ 0 роздумує...
Філософ 4 їсть (з запрошенням)
Філософ 2 закінчив їсти
Філософ 2 роздумує...
Філософ 1 їсть (БЕЗ запрошення)
Філософ 4 закінчив їсти та повернув запрошення
Філософ 4 роздумує...
Філософ 3 отримав запрошення
Філософ 3 їсть (з запрошенням)
Філософ 1 закінчив їсти
Філософ 1 роздумує...
Філософ 3 закінчив їсти та повернув запрошення
Філософ 3 роздумує...
Філософ 2 отримав запрошення
Філософ 2 їсть (з запрошенням)
Філософ 2 закінчив їсти та повернув запрошення
Філософ 2 роздумує...
Філософ 1 отримав запрошення
Філософ 1 їсть (з запрошенням)
Філософ 3 їсть (БЕЗ запрошення)
Філософ 1 закінчив їсти та повернув запрошення
Філософ 1 роздумує...
Філософ 4 отримав запрошення
Філософ 3 закінчив їсти
Філософ 3 роздумує...
Філософ 4 їсть (з запрошенням)
Фі

5. Побудувати модель взаємодії процесів у комп'ютері, що має два процесори. магнітний диск 
та принтер.


In [69]:
class ComputerSystemPN:
    def __init__(self):
        self.pn = PetriNet()
        
        # Позиції
        self.pn.add_place("cpu1_free", 1)
        self.pn.add_place("cpu2_free", 1)
        self.pn.add_place("disk_free", 1)
        self.pn.add_place("printer_free", 1)
        self.pn.add_place("task_queue", 5)  # Черга задач
        self.pn.add_place("cpu1_working", 0)
        self.pn.add_place("cpu2_working", 0)
        self.pn.add_place("disk_working", 0)
        self.pn.add_place("printing", 0)
        
        # Переходи для CPU1
        self.pn.add_transition("cpu1_start", [("cpu1_free", 1), ("task_queue", 1)], [("cpu1_working", 1)])
        self.pn.add_transition("cpu1_to_disk", [("cpu1_working", 1), ("disk_free", 1)], [("cpu1_free", 1), ("disk_working", 1)])
        self.pn.add_transition("disk_to_printer", [("disk_working", 1), ("printer_free", 1)], [("disk_free", 1), ("printing", 1)])
        self.pn.add_transition("printer_done", [("printing", 1)], [("printer_free", 1)])
        
        # Переходи для CPU2
        self.pn.add_transition("cpu2_start", [("cpu2_free", 1), ("task_queue", 1)], [("cpu2_working", 1)])
        self.pn.add_transition("cpu2_to_disk", [("cpu2_working", 1), ("disk_free", 1)], [("cpu2_free", 1), ("disk_working", 1)])
        
        self.tasks_completed = 0
        self.max_tasks = 5
    
    def cpu_worker(self, cpu_id):
        transition_start = f"cpu{cpu_id}_start"
        transition_to_disk = f"cpu{cpu_id}_to_disk"
        
        while self.tasks_completed < self.max_tasks:
            if self.pn.fire(transition_start):
                print(f"CPU{cpu_id}: розпочав обробку задачі")
                time.sleep(random.uniform(0.1, 0.2))
                
                while not self.pn.fire(transition_to_disk):
                    time.sleep(0.05)
                print(f"CPU{cpu_id}: передав дані на диск")
            else:
                time.sleep(0.05)
    
    def disk_worker(self):
        while self.tasks_completed < self.max_tasks:
            if self.pn.fire("disk_to_printer"):
                print("Диск: передав дані на принтер")
                time.sleep(random.uniform(0.1, 0.15))
            else:
                time.sleep(0.05)
    
    def printer_worker(self):
        while self.tasks_completed < self.max_tasks:
            if self.pn.fire("printer_done"):
                self.tasks_completed += 1
                print(f"Принтер: завершив друк ({self.tasks_completed}/{self.max_tasks})")
                time.sleep(random.uniform(0.1, 0.2))
            else:
                time.sleep(0.05)
    
    def run(self):
        threads = [
            threading.Thread(target=self.cpu_worker, args=(1,)),
            threading.Thread(target=self.cpu_worker, args=(2,)),
            threading.Thread(target=self.disk_worker),
            threading.Thread(target=self.printer_worker)
        ]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

cs = ComputerSystemPN()
cs.run()

CPU1: розпочав обробку задачі
CPU2: розпочав обробку задачі
CPU2: передав дані на диск
CPU2: розпочав обробку задачі
Диск: передав дані на принтер
Принтер: завершив друк (1/5)
CPU1: передав дані на диск
CPU1: розпочав обробку задачі
Диск: передав дані на принтер
Принтер: завершив друк (2/5)
CPU1: передав дані на диск
CPU1: розпочав обробку задачі
Диск: передав дані на принтер
CPU2: передав дані на диск
Принтер: завершив друк (3/5)
Диск: передав дані на принтер
CPU1: передав дані на диск
Принтер: завершив друк (4/5)
Диск: передав дані на принтер
Принтер: завершив друк (5/5)


6. Побудувати модель взаємодії трьох процесів, з яких один пише повідомлення в буфер, інші 
два обробляють повідомлення і поміщають результат у вихідний буфер

In [70]:
class ProducerConsumerPN:
    def __init__(self, buffer_size=5):
        self.pn = PetriNet()
        
        self.pn.add_place("writer_ready", 1)
        self.pn.add_place("input_buffer", 0)
        self.pn.add_place("processor1_ready", 1)
        self.pn.add_place("processor2_ready", 1)
        self.pn.add_place("output_buffer", 0)
        self.pn.add_place("buffer_space", buffer_size)
        
        self.pn.add_transition("write", [("writer_ready", 1), ("buffer_space", 1)], [("writer_ready", 1), ("input_buffer", 1)])
        self.pn.add_transition("process1", [("processor1_ready", 1), ("input_buffer", 1)], [("processor1_ready", 1), ("output_buffer", 1), ("buffer_space", 1)])
        self.pn.add_transition("process2", [("processor2_ready", 1), ("input_buffer", 1)], [("processor2_ready", 1), ("output_buffer", 1), ("buffer_space", 1)])
        
        self.messages_written = 0
        self.messages_processed = 0
        self.max_messages = 10
    
    def writer(self):
        for i in range(self.max_messages):
            while not self.pn.fire("write"):
                time.sleep(0.05)
            self.messages_written += 1
            print(f"Writer: написав повідомлення {i+1}")
            time.sleep(random.uniform(0.05, 0.1))
        print("Writer: завершив роботу")
    
    def processor(self, proc_id, transition_name):
        while self.messages_processed < self.max_messages:
            if self.pn.fire(transition_name):
                self.messages_processed += 1
                print(f"Processor {proc_id}: обробив повідомлення")
                time.sleep(random.uniform(0.05, 0.1))
            else:
                time.sleep(0.05)
        print(f"Processor {proc_id}: завершив роботу")
    
    def run(self):
        threads = [
            threading.Thread(target=self.writer),
            threading.Thread(target=self.processor, args=(1, "process1")),
            threading.Thread(target=self.processor, args=(2, "process2"))
        ]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

pc = ProducerConsumerPN(buffer_size=3)
pc.run()

Writer: написав повідомлення 1
Processor 1: обробив повідомлення
Writer: написав повідомлення 2
Processor 1: обробив повідомлення
Writer: написав повідомлення 3
Processor 1: обробив повідомлення
Writer: написав повідомлення 4
Processor 2: обробив повідомлення
Writer: написав повідомлення 5
Processor 2: обробив повідомлення
Writer: написав повідомлення 6
Processor 1: обробив повідомлення
Writer: написав повідомлення 7
Processor 1: обробив повідомлення
Writer: написав повідомлення 8
Processor 1: обробив повідомлення
Writer: написав повідомлення 9
Processor 1: обробив повідомлення
Writer: написав повідомлення 10
Processor 2: обробив повідомлення
Processor 1: завершив роботу
Writer: завершив роботу
Processor 2: завершив роботу


7. Перетворити модель завдання 6 при обмеженнях на розмір буферів.

In [71]:
class ProducerConsumerBufferLimitPN:
    def __init__(self, input_buffer_size=3, output_buffer_size=2):
        self.pn = PetriNet()
        
        self.pn.add_place("writer_ready", 1)
        self.pn.add_place("input_buffer", 0)
        self.pn.add_place("input_buffer_space", input_buffer_size)
        
        self.pn.add_place("processor1_ready", 1)
        self.pn.add_place("processor2_ready", 1)
        
        self.pn.add_place("output_buffer", 0)
        self.pn.add_place("output_buffer_space", output_buffer_size)
        
        # Writer пише в вхідний буфер (якщо є місце)
        self.pn.add_transition(
            "write",
            [("writer_ready", 1), ("input_buffer_space", 1)],
            [("writer_ready", 1), ("input_buffer", 1)]
        )
        
        # Processor 1 читає з вхідного буфера і пише у вихідний (якщо є місце)
        self.pn.add_transition(
            "process1",
            [("processor1_ready", 1), ("input_buffer", 1), ("output_buffer_space", 1)],
            [("processor1_ready", 1), ("input_buffer_space", 1), ("output_buffer", 1)]
        )
        
        # Processor 2 читає з вхідного буфера і пише у вихідний (якщо є місце)
        self.pn.add_transition(
            "process2",
            [("processor2_ready", 1), ("input_buffer", 1), ("output_buffer_space", 1)],
            [("processor2_ready", 1), ("input_buffer_space", 1), ("output_buffer", 1)]
        )
        
        # Читання з вихідного буфера
        self.pn.add_transition(
            "read_output",
            [("output_buffer", 1)],
            [("output_buffer_space", 1)]
        )
        
        self.messages_written = 0
        self.messages_processed = 0
        self.messages_read = 0
        self.max_messages = 10
    
    def writer(self):
        for i in range(self.max_messages):
            while not self.pn.fire("write"):
                time.sleep(0.05)
            self.messages_written += 1
            print(f"Writer: написав повідомлення {i+1} (вхідний буфер: {self.pn.places['input_buffer']})")
            time.sleep(random.uniform(0.05, 0.1))
        print("Writer: завершив роботу")
    
    def processor(self, proc_id, transition_name):
        while self.messages_processed < self.max_messages:
            if self.pn.fire(transition_name):
                self.messages_processed += 1
                print(f"Processor {proc_id}: обробив повідомлення (вихідний буфер: {self.pn.places['output_buffer']})")
                time.sleep(random.uniform(0.05, 0.15))
            else:
                time.sleep(0.05)
        print(f"Processor {proc_id}: завершив роботу")
    
    def reader(self):
        while self.messages_read < self.max_messages:
            if self.pn.fire("read_output"):
                self.messages_read += 1
                print(f"Reader: прочитав повідомлення {self.messages_read}")
                time.sleep(random.uniform(0.1, 0.2))
            else:
                time.sleep(0.05)
        print("Reader: завершив роботу")
    
    def run(self):
        threads = [
            threading.Thread(target=self.writer),
            threading.Thread(target=self.processor, args=(1, "process1")),
            threading.Thread(target=self.processor, args=(2, "process2")),
            threading.Thread(target=self.reader)
        ]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

pcb = ProducerConsumerBufferLimitPN()
pcb.run()

Writer: написав повідомлення 1 (вхідний буфер: 1)
Processor 1: обробив повідомлення (вихідний буфер: 1)
Reader: прочитав повідомлення 1
Writer: написав повідомлення 2 (вхідний буфер: 1)
Processor 2: обробив повідомлення (вихідний буфер: 1)
Writer: написав повідомлення 3 (вхідний буфер: 1)
Processor 2: обробив повідомлення (вихідний буфер: 2)
Reader: прочитав повідомлення 2
Writer: написав повідомлення 4 (вхідний буфер: 1)
Processor 1: обробив повідомлення (вихідний буфер: 2)
Writer: написав повідомлення 5 (вхідний буфер: 1)
Writer: написав повідомлення 6 (вхідний буфер: 2)
Reader: прочитав повідомлення 3
Processor 2: обробив повідомлення (вихідний буфер: 2)
Writer: написав повідомлення 7 (вхідний буфер: 2)
Writer: написав повідомлення 8 (вхідний буфер: 3)
Reader: прочитав повідомлення 4
Processor 1: обробив повідомлення (вихідний буфер: 2)
Writer: написав повідомлення 9 (вхідний буфер: 3)
Reader: прочитав повідомлення 5
Processor 1: обробив повідомлення (вихідний буфер: 2)
Writer: напи

8. Представити модель задачі 7 для довільного заданого числа процесів, що записують і 
зчитують повідомлення.

In [72]:
class ProducerConsumerMultiPN:
    def __init__(self, n_writers=2, n_processors=3, buffer_size=5):
        self.n_writers = n_writers
        self.n_processors = n_processors
        self.pn = PetriNet()
        
        # Позиції
        for i in range(n_writers):
            self.pn.add_place(f"writer_{i}_ready", 1)
        
        self.pn.add_place("input_buffer", 0)
        self.pn.add_place("buffer_space", buffer_size)
        
        for i in range(n_processors):
            self.pn.add_place(f"processor_{i}_ready", 1)
        
        self.pn.add_place("output_buffer", 0)
        
        # Переходи дляwriters
        for i in range(n_writers):
            self.pn.add_transition(
                f"write_{i}",
                [(f"writer_{i}_ready", 1), ("buffer_space", 1)],
                [(f"writer_{i}_ready", 1), ("input_buffer", 1)]
            )
        
        # Переходи для processors
        for i in range(n_processors):
            self.pn.add_transition(
                f"process_{i}",
                [(f"processor_{i}_ready", 1), ("input_buffer", 1)],
                [(f"processor_{i}_ready", 1), ("output_buffer", 1), ("buffer_space", 1)]
            )
        
        self.messages_written = 0
        self.messages_processed = 0
        self.max_messages = 15
        self.lock = threading.Lock()
    
    def writer(self, writer_id):
        messages = self.max_messages // self.n_writers
        for i in range(messages):
            while not self.pn.fire(f"write_{writer_id}"):
                time.sleep(0.05)
            with self.lock:
                self.messages_written += 1
            print(f"Writer {writer_id}: написав повідомлення {i+1}")
            time.sleep(random.uniform(0.05, 0.1))
        print(f"Writer {writer_id}: завершив роботу")
    
    def processor(self, proc_id):
        while True:
            with self.lock:
                if self.messages_processed >= self.max_messages:
                    break
            
            if self.pn.fire(f"process_{proc_id}"):
                with self.lock:
                    self.messages_processed += 1
                    count = self.messages_processed
                print(f"Processor {proc_id}: обробив повідомлення ({count}/{self.max_messages})")
                time.sleep(random.uniform(0.05, 0.15))
            else:
                time.sleep(0.05)
        print(f"Processor {proc_id}: завершив роботу")
    
    def run(self):
        threads = []
        
        # Створити writers
        for i in range(self.n_writers):
            threads.append(threading.Thread(target=self.writer, args=(i,)))
        
        # Створити processors
        for i in range(self.n_processors):
            threads.append(threading.Thread(target=self.processor, args=(i,)))
        
        for t in threads:
            t.start()
        for t in threads:
            t.join()

pcm = ProducerConsumerMultiPN()

9. Побудувати модель ліфта що перевозить людей для чотириповерхового будинку. Кнопки виклику знаходяться на кожному поверсі.

In [73]:
import random
import time


class PetriNetEl:
    def __init__(self):
        self.places = {}
        self.transitions = {}

    def add_place(self, name, tokens=0):
        self.places[name] = tokens

    def add_transition(self, name, inputs, outputs):
        self.transitions[name] = (inputs, outputs)

    def enabled(self, t):
        inputs, _ = self.transitions[t]
        return all(self.places[p] >= w for p, w in inputs.items())

    def fire(self, t):
        if not self.enabled(t):
            return False
        inputs, outputs = self.transitions[t]
        for p, w in inputs.items():
            self.places[p] -= w
        for p, w in outputs.items():
            self.places[p] += w
        return True

class ElevatorModel:
    def __init__(self, floors=4, capacity=3):
        self.floors = floors
        self.capacity = capacity

        # Мережа Петрі
        self.net = PetriNetEl()

        # ліфт на кожному поверсі
        for f in range(floors):
            self.net.add_place(f"lift_at_{f}", tokens=1 if f == 0 else 0)

        # Пасажири в ліфті, розділені по цілях
        for f in range(floors):
            self.net.add_place(f"in_lift_to_{f}", 0)

        # Очікування пасажирів на поверсі -> з цілями
        for f in range(floors):
            for t in range(floors):
                if f != t:
                    self.net.add_place(f"wait_{f}_to_{t}", 0)

        self.net.add_place("can_move", 1)

        # переходи: рух вгору / вниз
        for f in range(floors - 1):
            self.net.add_transition(
                f"move_up_{f}_{f+1}",
                inputs={f"lift_at_{f}": 1, "can_move": 1},
                outputs={f"lift_at_{f+1}": 1, "can_move": 1}
            )
            self.net.add_transition(
                f"move_down_{f+1}_{f}",
                inputs={f"lift_at_{f+1}": 1, "can_move": 1},
                outputs={f"lift_at_{f}": 1, "can_move": 1}
            )

        # Посадка: wait_f_to_t -> in_lift_to_t
        for f in range(floors):
            for t in range(floors):
                if f != t:
                    self.net.add_transition(
                        f"board_{f}_to_{t}",
                        inputs={f"lift_at_{f}": 1, f"wait_{f}_to_{t}": 1},
                        outputs={f"lift_at_{f}": 1, f"in_lift_to_{t}": 1},
                    )

        # Висадка
        for f in range(floors):
            self.net.add_transition(
                f"exit_{f}",
                inputs={f"lift_at_{f}": 1, f"in_lift_to_{f}": 1},
                outputs={f"lift_at_{f}": 1},
            )

    def add_random_passengers(self, count):
        for _ in range(count):
            start = random.randint(0, self.floors - 1)
            target = random.choice([f for f in range(self.floors) if f != start])
            self.net.places[f"wait_{start}_to_{target}"] += 1
            print(f"[ВИКЛИК] Поверх {start}: пасажир хоче на {target}")

    def lift_load(self):
        return sum(self.net.places[f"in_lift_to_{t}"] for t in range(self.floors))

    def current_load(self):
        total = 0
        for t in range(self.floors):
            total += self.net.places[f"in_lift_to_{t}"]
        return total

    def load_per_target(self):
        d = {}
        for t in range(self.floors):
            c = self.net.places[f"in_lift_to_{t}"]
            if c > 0:
                d[t] = c
        return d
   
    def step(self):

        for f in range(self.floors):
            t = f"exit_{f}"
            if self.net.enabled(t):
                self.net.fire(t)
                print(f"[ВИХІД] Поверх {f}: пасажир вийшов")
                return True

        if self.lift_load() < self.capacity:
            for f in range(self.floors):
                for t in range(self.floors):
                    if f != t:
                        tr = f"board_{f}_to_{t}"
                        if self.net.enabled(tr):
                            self.net.fire(tr)
                            print(f"[ПОСАДКА] Поверх {f}: пасажир сідає → {t}. У ліфті тепер {self.current_load()} пасажирів. Цілі: {self.load_per_target()}")

                            return True

        # поточний поверх
        current_floor = None
        for f in range(self.floors):
            if self.net.places[f"lift_at_{f}"] == 1:
                current_floor = f
                break

        # перевіряємо, чи є зверху хтось
        def has_tasks_above():
            for f in range(current_floor + 1, self.floors):
                # пасажири чекають
                for t in range(self.floors):
                    if f != t and self.net.places[f"wait_{f}_to_{t}"] > 0:
                        return True
                # пасажири в ліфті їдуть наверх
                if self.net.places[f"in_lift_to_{f}"] > 0:
                    return True
            return False

        # перевіряємо, чи є знизу хтось
        def has_tasks_below():
            for f in range(0, current_floor):
                for t in range(self.floors):
                    if f != t and self.net.places[f"wait_{f}_to_{t}"] > 0:
                        return True
                if self.net.places[f"in_lift_to_{f}"] > 0:
                    return True
            return False

        # правила руху:
        # 1) якщо є задачі нагорі → рух вгору
        # 2) інакше якщо є задачі внизу → рух вниз
        # 3) інакше зупинка

        # рух вгору
        if has_tasks_above() and current_floor < self.floors - 1:
            t = f"move_up_{current_floor}_{current_floor+1}"
            if self.net.enabled(t):
                self.net.fire(t)
                print(f"[РУХ] Ліфт {current_floor} → {current_floor+1}")
                return True

        # рух вниз
        if has_tasks_below() and current_floor > 0:
            t = f"move_down_{current_floor}_{current_floor-1}"
            if self.net.enabled(t):
                self.net.fire(t)
                print(f"[РУХ] Ліфт {current_floor} → {current_floor-1}")
                return True

        print("Немає задач — ліфт стоїть.")
        return False

    def run(self, steps=50):
        for _ in range(steps):
            if not self.step():
                print("Модель зупинилася — немає доступних переходів.")
                break
            time.sleep(0.3)


model = ElevatorModel(floors=4, capacity=1000)

model.add_random_passengers(10)

model.run(steps=60)


[ВИКЛИК] Поверх 1: пасажир хоче на 3
[ВИКЛИК] Поверх 0: пасажир хоче на 2
[ВИКЛИК] Поверх 1: пасажир хоче на 2
[ВИКЛИК] Поверх 1: пасажир хоче на 3
[ВИКЛИК] Поверх 3: пасажир хоче на 0
[ВИКЛИК] Поверх 1: пасажир хоче на 2
[ВИКЛИК] Поверх 2: пасажир хоче на 3
[ВИКЛИК] Поверх 2: пасажир хоче на 0
[ВИКЛИК] Поверх 1: пасажир хоче на 0
[ВИКЛИК] Поверх 0: пасажир хоче на 3
[ПОСАДКА] Поверх 0: пасажир сідає → 2. У ліфті тепер 1 пасажирів. Цілі: {2: 1}
[ПОСАДКА] Поверх 0: пасажир сідає → 3. У ліфті тепер 2 пасажирів. Цілі: {2: 1, 3: 1}
[РУХ] Ліфт 0 → 1
[ПОСАДКА] Поверх 1: пасажир сідає → 0. У ліфті тепер 3 пасажирів. Цілі: {0: 1, 2: 1, 3: 1}
[ПОСАДКА] Поверх 1: пасажир сідає → 2. У ліфті тепер 4 пасажирів. Цілі: {0: 1, 2: 2, 3: 1}
[ПОСАДКА] Поверх 1: пасажир сідає → 2. У ліфті тепер 5 пасажирів. Цілі: {0: 1, 2: 3, 3: 1}
[ПОСАДКА] Поверх 1: пасажир сідає → 3. У ліфті тепер 6 пасажирів. Цілі: {0: 1, 2: 3, 3: 2}
[ПОСАДКА] Поверх 1: пасажир сідає → 3. У ліфті тепер 7 пасажирів. Цілі: {0: 1, 2: 3, 

10. Побудувати модель задачі 9 з обмеженнями на кількість пасажирів ліфту.

In [74]:
model = ElevatorModel(floors=4, capacity=3)

model.add_random_passengers(10)

model.run(steps=60)

[ВИКЛИК] Поверх 2: пасажир хоче на 1
[ВИКЛИК] Поверх 0: пасажир хоче на 1
[ВИКЛИК] Поверх 1: пасажир хоче на 3
[ВИКЛИК] Поверх 3: пасажир хоче на 1
[ВИКЛИК] Поверх 0: пасажир хоче на 2
[ВИКЛИК] Поверх 0: пасажир хоче на 2
[ВИКЛИК] Поверх 1: пасажир хоче на 3
[ВИКЛИК] Поверх 1: пасажир хоче на 3
[ВИКЛИК] Поверх 2: пасажир хоче на 0
[ВИКЛИК] Поверх 0: пасажир хоче на 1
[ПОСАДКА] Поверх 0: пасажир сідає → 1. У ліфті тепер 1 пасажирів. Цілі: {1: 1}
[ПОСАДКА] Поверх 0: пасажир сідає → 1. У ліфті тепер 2 пасажирів. Цілі: {1: 2}
[ПОСАДКА] Поверх 0: пасажир сідає → 2. У ліфті тепер 3 пасажирів. Цілі: {1: 2, 2: 1}
[РУХ] Ліфт 0 → 1
[ВИХІД] Поверх 1: пасажир вийшов
[ВИХІД] Поверх 1: пасажир вийшов
[ПОСАДКА] Поверх 1: пасажир сідає → 3. У ліфті тепер 2 пасажирів. Цілі: {2: 1, 3: 1}
[ПОСАДКА] Поверх 1: пасажир сідає → 3. У ліфті тепер 3 пасажирів. Цілі: {2: 1, 3: 2}
[РУХ] Ліфт 1 → 2
[ВИХІД] Поверх 2: пасажир вийшов
[ПОСАДКА] Поверх 2: пасажир сідає → 0. У ліфті тепер 3 пасажирів. Цілі: {0: 1, 3: 2}

11. Побудувати модель складання іспиту групою із чотирьох студентів одному екзаменатору.

In [75]:
# ==================== 11. Екзаменатор і студенти ====================
class ExamPN:
    def __init__(self, n_students=4):
        self.n = n_students
        self.pn = PetriNet()
        
        self.pn.add_place("examiner_free", 1)
        for i in range(n_students):
            self.pn.add_place(f"student_{i}_waiting", 1)
            self.pn.add_place(f"student_{i}_examining", 0)
            self.pn.add_place(f"student_{i}_done", 0)
        
        for i in range(n_students):
            self.pn.add_transition(
                f"start_exam_{i}",
                [(f"student_{i}_waiting", 1), ("examiner_free", 1)],
                [(f"student_{i}_examining", 1)]
            )
            
            self.pn.add_transition(
                f"finish_exam_{i}",
                [(f"student_{i}_examining", 1)],
                [(f"student_{i}_done", 1), ("examiner_free", 1)]
            )
    
    def student(self, student_id):
        print(f"Студент {student_id}: готується до іспиту")
        time.sleep(random.uniform(0.1, 0.2))
        
        print(f"Студент {student_id}: чекає на екзаменатора")
        while not self.pn.fire(f"start_exam_{student_id}"):
            time.sleep(0.05)
        
        print(f"Студент {student_id}: складає іспит")
        time.sleep(random.uniform(0.3, 0.5))
        
        self.pn.fire(f"finish_exam_{student_id}")
        grade = random.randint(60, 100)
        print(f"Студент {student_id}: отримав оцінку {grade}")
    
    def run(self):
        threads = [threading.Thread(target=self.student, args=(i,)) for i in range(self.n)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

exam = ExamPN(4)
exam.run()

Студент 0: готується до іспиту
Студент 1: готується до іспиту
Студент 2: готується до іспиту
Студент 3: готується до іспиту
Студент 0: чекає на екзаменатора
Студент 0: складає іспит
Студент 1: чекає на екзаменатора
Студент 2: чекає на екзаменатора
Студент 3: чекає на екзаменатора
Студент 0: отримав оцінку 62Студент 3: складає іспит

Студент 3: отримав оцінку 76
Студент 1: складає іспит
Студент 1: отримав оцінку 89
Студент 2: складає іспит
Студент 2: отримав оцінку 64


12. Побудувати модель перукарні, в якій працює три перукарі та є п'ять сидячих місць у 
кімнаті для очікування клієнтів.

In [76]:
# ==================== 12. Перукарня ====================
class BarberShopPN:
    def __init__(self):
        self.pn = PetriNet()
        
        # 3 перукарі, 5 місць для очікування
        self.pn.add_place("barber1_free", 1)
        self.pn.add_place("barber2_free", 1)
        self.pn.add_place("barber3_free", 1)
        self.pn.add_place("waiting_seats", 5)
        self.pn.add_place("customers_waiting", 0)
        
        for i in range(1, 4):
            self.pn.add_transition(
                f"barber{i}_start",
                [(f"barber{i}_free", 1), ("customers_waiting", 1)],
                []  # Перукар зайнятий, клієнт обслуговується
            )
            
            self.pn.add_transition(
                f"barber{i}_finish",
                [],
                [(f"barber{i}_free", 1), ("waiting_seats", 1)]
            )
        
        self.customers_served = 0
        self.max_customers = 8
    
    def customer(self, cust_id):
        print(f"Клієнт {cust_id}: прийшов до перукарні")
        
        # Спроба зайняти місце
        if self.pn.places["waiting_seats"] > 0:
            with self.pn.lock:
                if self.pn.places["waiting_seats"] > 0:
                    self.pn.places["waiting_seats"] -= 1
                    self.pn.places["customers_waiting"] += 1
                    print(f"Клієнт {cust_id}: сів чекати")
                else:
                    print(f"Клієнт {cust_id}: немає місць, пішов")
                    return
        else:
            print(f"Клієнт {cust_id}: немає місць, пішов")
            return
        
        # Чекаємо на перукаря
        served = False
        for i in range(1, 4):
            if self.pn.fire(f"barber{i}_start"):
                print(f"Клієнт {cust_id}: обслуговує перукар {i}")
                time.sleep(random.uniform(0.3, 0.5))
                self.pn.fire(f"barber{i}_finish")
                print(f"Клієнт {cust_id}: підстрижений")
                self.customers_served += 1
                served = True
                break
            
        if not served:
            time.sleep(0.1)
    
    def run(self):
        threads = [threading.Thread(target=self.customer, args=(i,)) for i in range(self.max_customers)]
        for t in threads:
            t.start()
            time.sleep(random.uniform(0.1, 0.3))
        for t in threads:
            t.join()

barber = BarberShopPN()
barber.run()

Клієнт 0: прийшов до перукарні
Клієнт 0: сів чекати
Клієнт 0: обслуговує перукар 1
Клієнт 1: прийшов до перукарні
Клієнт 1: сів чекати
Клієнт 1: обслуговує перукар 2
Клієнт 2: прийшов до перукарні
Клієнт 2: сів чекати
Клієнт 2: обслуговує перукар 3
Клієнт 0: підстрижений
Клієнт 3: прийшов до перукарні
Клієнт 3: сів чекати
Клієнт 3: обслуговує перукар 1
Клієнт 1: підстрижений
Клієнт 4: прийшов до перукарні
Клієнт 4: сів чекати
Клієнт 4: обслуговує перукар 2
Клієнт 2: підстрижений
Клієнт 5: прийшов до перукарні
Клієнт 5: сів чекати
Клієнт 5: обслуговує перукар 3
Клієнт 3: підстрижений
Клієнт 6: прийшов до перукарні
Клієнт 6: сів чекати
Клієнт 6: обслуговує перукар 1
Клієнт 4: підстрижений
Клієнт 5: підстрижений
Клієнт 7: прийшов до перукарні
Клієнт 7: сів чекати
Клієнт 7: обслуговує перукар 2
Клієнт 6: підстрижений
Клієнт 7: підстрижений


13. Побудувати модель завдання 12 для випадку, коли клієнт, якому не дістається місце, може 
або чекати стоячи, або піти непідстриженим.

In [77]:
class BarberShopStandingPN:
    def __init__(self):
        self.pn = PetriNet()
        
        self.pn.add_place("barber1_free", 1)
        self.pn.add_place("barber2_free", 1)
        self.pn.add_place("barber3_free", 1)
        self.pn.add_place("waiting_seats", 5)
        self.pn.add_place("customers_sitting", 0)
        self.pn.add_place("customers_standing", 0)
        
        for i in range(1, 4):
            # Обслуговування клієнтів, що сидять
            self.pn.add_transition(
                f"barber{i}_start_sitting",
                [(f"barber{i}_free", 1), ("customers_sitting", 1)],
                []
            )
            
            # Обслуговування клієнтів, що стоять
            self.pn.add_transition(
                f"barber{i}_start_standing",
                [(f"barber{i}_free", 1), ("customers_standing", 1)],
                []
            )
            
            self.pn.add_transition(
                f"barber{i}_finish",
                [],
                [(f"barber{i}_free", 1), ("waiting_seats", 1)]
            )
        
        self.customers_served = 0
        self.customers_left = 0
        self.max_customers = 12
    
    def customer(self, cust_id):
        print(f"Клієнт {cust_id}: прийшов до перукарні")
        
        # Спроба зайняти місце
        sitting = False
        standing = False
        
        if self.pn.places["waiting_seats"] > 0:
            with self.pn.lock:
                if self.pn.places["waiting_seats"] > 0:
                    self.pn.places["waiting_seats"] -= 1
                    self.pn.places["customers_sitting"] += 1
                    sitting = True
                    print(f"Клієнт {cust_id}: сів чекати")
        
        if not sitting:
            # Вирішує чекати стоячи або піти
            wait_standing = random.choice([True, False])
            
            if wait_standing:
                with self.pn.lock:
                    self.pn.places["customers_standing"] += 1
                    standing = True
                print(f"Клієнт {cust_id}: немає місць, чекає стоячи")
            else:
                print(f"Клієнт {cust_id}: немає місць, пішов непідстриженим")
                self.customers_left += 1
                return
        
        # Чекаємо на перукаря
        served = False
        for _ in range(50):  # Максимум спроб
            for i in range(1, 4):
                transition = f"barber{i}_start_sitting" if sitting else f"barber{i}_start_standing"
                if self.pn.fire(transition):
                    print(f"Клієнт {cust_id}: обслуговує перукар {i}")
                    time.sleep(random.uniform(0.3, 0.5))
                    self.pn.fire(f"barber{i}_finish")
                    print(f"Клієнт {cust_id}: підстрижений")
                    self.customers_served += 1
                    served = True
                    break
            
            if served:
                break
            time.sleep(0.1)
    
    def run(self):
        threads = [threading.Thread(target=self.customer, args=(i,)) for i in range(self.max_customers)]
        for t in threads:
            t.start()
            time.sleep(random.uniform(0.05, 0.15))
        for t in threads:
            t.join()
        
        print(f"\nПідсумок: обслуговано {self.customers_served}, пішли {self.customers_left}")

barberstand = BarberShopStandingPN()
barberstand.run()

Клієнт 0: прийшов до перукарні
Клієнт 0: сів чекати
Клієнт 0: обслуговує перукар 1
Клієнт 1: прийшов до перукарні
Клієнт 1: сів чекати
Клієнт 1: обслуговує перукар 2
Клієнт 2: прийшов до перукарні
Клієнт 2: сів чекати
Клієнт 2: обслуговує перукар 3
Клієнт 3: прийшов до перукарні
Клієнт 3: сів чекати
Клієнт 4: прийшов до перукарні
Клієнт 4: сів чекати
Клієнт 1: підстрижений
Клієнт 0: підстрижений
Клієнт 3: обслуговує перукар 1
Клієнт 4: обслуговує перукар 2
Клієнт 2: підстрижений
Клієнт 5: прийшов до перукарні
Клієнт 5: сів чекати
Клієнт 5: обслуговує перукар 3
Клієнт 6: прийшов до перукарні
Клієнт 6: сів чекати
Клієнт 7: прийшов до перукарні
Клієнт 7: сів чекати
Клієнт 8: прийшов до перукарні
Клієнт 8: немає місць, пішов непідстриженим
Клієнт 9: прийшов до перукарні
Клієнт 9: немає місць, чекає стоячи
Клієнт 5: підстрижений
Клієнт 3: підстрижений
Клієнт 4: підстрижений
Клієнт 9: обслуговує перукар 1
Клієнт 7: обслуговує перукар 2
Клієнт 6: обслуговує перукар 3
Клієнт 10: прийшов до пер

<img src="14_17.png">

14-17. Побудувати модель безпечного руху на ділянці залізниці, що представлена на схемі. 
Рух односторонній. Дорога складається з перегонів зі світлофорами (8) та датчиками зайнятості

In [78]:
class RailwayPN:
    def __init__(self, segments=4):
        self.segments = segments
        self.pn = PetriNet()
        
        # Позиції для кожного сегмента
        for i in range(segments):
            self.pn.add_place(f"segment_{i}_free", 1)
            self.pn.add_place(f"train_at_{i}", 0)
            self.pn.add_place(f"signal_{i}_green", 1 if i == 0 else 0)
        
        # Початкова позиція поїзда
        self.pn.add_place("train_waiting", 1)
        
        # Перехід входу на перший сегмент
        self.pn.add_transition(
            "enter_0",
            [("train_waiting", 1), ("segment_0_free", 1), ("signal_0_green", 1)],
            [("train_at_0", 1)]
        )
        
        # Переходи між сегментами
        for i in range(segments - 1):
            self.pn.add_transition(
                f"move_{i}_to_{i+1}",
                [(f"train_at_{i}", 1), (f"segment_{i+1}_free", 1), (f"signal_{i+1}_green", 1)],
                [(f"train_at_{i+1}", 1), (f"segment_{i}_free", 1), (f"signal_{i+1}_green", 0)]
            )
        
        # Вихід з останнього сегмента
        self.pn.add_transition(
            f"exit_{segments-1}",
            [(f"train_at_{segments-1}", 1)],
            [(f"segment_{segments-1}_free", 1), ("train_waiting", 1)]
        )
    
    def signal_controller(self):
        """Контролер світлофорів"""
        while True:
            for i in range(self.segments):
                with self.pn.lock:
                    # Якщо сегмент вільний і поїзда немає, увімкнути зелений
                    if self.pn.places[f"segment_{i}_free"] > 0 and self.pn.places[f"train_at_{i}"] == 0:
                        self.pn.places[f"signal_{i}_green"] = 1
                    else:
                        self.pn.places[f"signal_{i}_green"] = 0
            time.sleep(0.1)
    
    def train(self, train_id):
        print(f"Поїзд {train_id}: очікує на платформі")
        
        # Вхід на перший сегмент
        while not self.pn.fire("enter_0"):
            time.sleep(0.1)
        print(f"Поїзд {train_id}: увійшов на сегмент 0")
        time.sleep(0.3)
        
        # Рух по сегментах
        for i in range(self.segments - 1):
            while not self.pn.fire(f"move_{i}_to_{i+1}"):
                time.sleep(0.1)
            print(f"Поїзд {train_id}: перейшов з сегмента {i} на {i+1}")
            time.sleep(0.3)
        
        # Вихід
        self.pn.fire(f"exit_{self.segments-1}")
        print(f"Поїзд {train_id}: залишив дільницю")
    
    def run(self, num_trains=3):
        controller = threading.Thread(target=self.signal_controller, daemon=True)
        controller.start()
        
        threads = []
        for i in range(num_trains+1):
            t = threading.Thread(target=self.train, args=(i,))
            threads.append(t)
            t.start()
            time.sleep(1.0)  # Затримка між поїздами
        
        for t in threads:
            t.join()

rw = RailwayPN(7)
rw.run(4)

Поїзд 0: очікує на платформі
Поїзд 0: увійшов на сегмент 0
Поїзд 0: перейшов з сегмента 0 на 1
Поїзд 0: перейшов з сегмента 1 на 2
Поїзд 0: перейшов з сегмента 2 на 3
Поїзд 1: очікує на платформі
Поїзд 0: перейшов з сегмента 3 на 4
Поїзд 0: перейшов з сегмента 4 на 5
Поїзд 0: перейшов з сегмента 5 на 6
Поїзд 2: очікує на платформі
Поїзд 0: залишив дільницю
Поїзд 1: увійшов на сегмент 0
Поїзд 1: перейшов з сегмента 0 на 1
Поїзд 1: перейшов з сегмента 1 на 2
Поїзд 3: очікує на платформі
Поїзд 1: перейшов з сегмента 2 на 3
Поїзд 1: перейшов з сегмента 3 на 4
Поїзд 1: перейшов з сегмента 4 на 5
Поїзд 1: перейшов з сегмента 5 на 6
Поїзд 4: очікує на платформі
Поїзд 1: залишив дільницю
Поїзд 2: увійшов на сегмент 0
Поїзд 2: перейшов з сегмента 0 на 1
Поїзд 2: перейшов з сегмента 1 на 2
Поїзд 2: перейшов з сегмента 2 на 3
Поїзд 2: перейшов з сегмента 3 на 4
Поїзд 2: перейшов з сегмента 4 на 5
Поїзд 2: перейшов з сегмента 5 на 6
Поїзд 2: залишив дільницю
Поїзд 3: увійшов на сегмент 0
Поїзд 3: 

18. Побудувати модель ядерной реакції, у якій з кожного атому, що стикається з нейтроном, 
виділяється при розпаді вісім нейтронів і створюється новий атом.

In [79]:
class NuclearReactionPN:
    def __init__(self, initial_atoms=3, neutrons_per_reaction=8):
        self.pn = PetriNet()
        self.neutrons_per_reaction = neutrons_per_reaction
        
        self.pn.add_place("atoms", initial_atoms)
        self.pn.add_place("neutrons", 1)  # Початковий нейтрон
        self.pn.add_place("new_atoms", 0)
        
        # Реакція: атом + нейтрон -> новий атом + 8 нейтронів
        self.pn.add_transition(
            "reaction",
            [("atoms", 1), ("neutrons", 1)],
            [("new_atoms", 1), ("neutrons", neutrons_per_reaction)]
        )
        
        self.reactions_count = 0
        self.max_reactions = 10
        self.lock = threading.Lock()
    
    def reaction_monitor(self):
        print(f"Початкова кількість атомів: {self.pn.places['atoms']}")
        print(f"Початкова кількість нейтронів: {self.pn.places['neutrons']}")
        print()
        
        while self.reactions_count < self.max_reactions:
            if self.pn.fire("reaction"):
                with self.lock:
                    self.reactions_count += 1
                
                atoms = self.pn.places['atoms']
                neutrons = self.pn.places['neutrons']
                new_atoms = self.pn.places['new_atoms']
                
                print(f"Реакція #{self.reactions_count}:")
                print(f"  Залишилось старих атомів: {atoms}")
                print(f"  Утворилось нових атомів: {new_atoms}")
                print(f"  Кількість нейтронів: {neutrons}")
                print()
                
                time.sleep(0.2)
            else:
                # Якщо реакція неможлива (немає атомів або нейтронів)
                atoms = self.pn.places['atoms']
                neutrons = self.pn.places['neutrons']
                
                if neutrons > 0 and atoms == 0:
                    print("Реакція припинилася: закінчилися атоми")
                elif neutrons == 0 and atoms > 0:
                    print("Реакція припинилася: закінчилися нейтрони")
                else:
                    print("Реакція припинилася")
                break
            
            # Обмеження для запобігання експоненціальному зростанню
            if self.pn.places['neutrons'] > 100:
                print("Критична маса нейтронів досягнута! Аварійна зупинка.")
                break
        
        print(f"\nФінальний стан:")
        print(f"  Старих атомів: {self.pn.places['atoms']}")
        print(f"  Нових атомів: {self.pn.places['new_atoms']}")
        print(f"  Нейтронів: {self.pn.places['neutrons']}")
        print(f"  Всього реакцій: {self.reactions_count}")
    
    def run(self):
        monitor = threading.Thread(target=self.reaction_monitor)
        monitor.start()
        monitor.join()

nr = NuclearReactionPN(5)
nr.run()

Початкова кількість атомів: 5
Початкова кількість нейтронів: 1

Реакція #1:
  Залишилось старих атомів: 4
  Утворилось нових атомів: 1
  Кількість нейтронів: 8

Реакція #2:
  Залишилось старих атомів: 3
  Утворилось нових атомів: 2
  Кількість нейтронів: 15

Реакція #3:
  Залишилось старих атомів: 2
  Утворилось нових атомів: 3
  Кількість нейтронів: 22

Реакція #4:
  Залишилось старих атомів: 1
  Утворилось нових атомів: 4
  Кількість нейтронів: 29

Реакція #5:
  Залишилось старих атомів: 0
  Утворилось нових атомів: 5
  Кількість нейтронів: 36

Реакція припинилася: закінчилися атоми

Фінальний стан:
  Старих атомів: 0
  Нових атомів: 5
  Нейтронів: 36
  Всього реакцій: 5
